# Imviz profiling

Create three channels of smoothly varying images:

In [ ]:
%%time
from time import sleep

import numpy as np
from scipy.ndimage import gaussian_filter
from astropy.nddata import NDData
from astropy.wcs import WCS

from jdaviz import Imviz

shape = (4000, 4000)

wcs = WCS({
    'CTYPE1': 'RA---TAN', 
    'CUNIT1': 'deg', 
    'CDELT1': 0.01,
    'CRPIX1': shape[0] / 2, 
    'CRVAL1': 180,
    'CTYPE2': 'DEC--TAN', 
    'CUNIT2': 'deg', 
    'CDELT2': 0.01,
    'CRPIX2': shape[1] / 2, 
    'CRVAL2': 0
})

random_shape = (6, 6)
image_c = np.random.uniform(size=random_shape)
image_y = np.random.uniform(size=random_shape)
image_m = np.random.uniform(size=random_shape)

colors = ['c', 'y', 'm']

def low_pass_filter(x):
    rfft = np.fft.rfft2(x)
    irfft = np.fft.irfft2(rfft, shape)
    return 2**16 * irfft / irfft.max()

nddata_cym = [
    NDData(low_pass_filter(image), wcs=wcs) 
    for image in [image_c, image_y, image_m]
]

#### Profiling result for cell 0: 
No output.


Initialize and show Imviz:

In [ ]:
%%time

imviz = Imviz()
imviz.app.layout.border = '2px solid rgb(143, 56, 3)'

label_mouseover = imviz.app.session.application._tools['g-coords-info']
viewer = imviz.default_viewer._obj

# imviz.show('sidecar:split-right', height=800)
imviz.show(height=800)

![output screenshot](output-003-2024-11-07T20-32-29.960909.png)

#### Profiling result for cell 1: 
* 1.41 seconds elapsed
 * 3 output updates


You can optionally set the compression to one of these two settings:

In [ ]:
# imviz.app.get_viewer('imviz-0')._composite_image.compression = 'png'

# imviz.app.get_viewer('imviz-0')._composite_image.compression = 'none'

#### Profiling result for cell 2: 
No output.


Load the images into Imviz:

In [ ]:
%%time

with imviz.batch_load():
    for nddata, data_label in zip(nddata_cym, colors):
        imviz.load_data(nddata, data_label=data_label)

compression = imviz.app.get_viewer('imviz-0')._composite_image.compression
print(f"glue-jupyter compression configuration: '{compression}'")

![output screenshot](output-003-2024-11-07T20-32-50.264281.png)

#### Profiling result for cell 3: 
* 1.39 seconds elapsed
 * 4 output updates


Use WCS to align the images, zoom to fit:

In [ ]:
%%time

orientation = imviz.plugins['Orientation']
orientation.align_by = 'WCS'
viewer.zoom_level = 'fit'

![output screenshot](output-003-2024-11-07T20-33-00.925076.png)

#### Profiling result for cell 4: 
* 1.92 seconds elapsed
 * 7 output updates


Rotate the image orientation by 180 deg counter-clockwise:

In [ ]:
%%time

# rotate the image by 180 deg (CCW):
orientation.rotation_angle = 180
orientation.add_orientation()

![output screenshot](output-003-2024-11-07T20-33-11.249760.png)

#### Profiling result for cell 5: 
* 2.12 seconds elapsed
 * 6 output updates


Assign colors to each of the layers:

In [ ]:
%%time

plot_options = imviz.plugins["Plot Options"]
for layer, color in zip(plot_options.layer.choices, colors):
    plot_options.layer = layer
    plot_options.image_color_mode = 'Color'
    plot_options.image_color = color
    plot_options.image_opacity = 0.7

![output screenshot](output-003-2024-11-07T20-33-21.626969.png)

#### Profiling result for cell 6: 
* 2.30 seconds elapsed
 * 5 output updates


Return to the default orientation:

In [ ]:
%%time

orientation.orientation = 'Default orientation'

![output screenshot](output-003-2024-11-07T20-33-31.652719.png)

#### Profiling result for cell 7: 
* 2.14 seconds elapsed
 * 3 output updates


Blink through each layer (one visible at a time):

In [ ]:
%%time

for i in range(len(plot_options.layer.choices)):
    viewer.blink_once()

![output screenshot](output-003-2024-11-07T20-33-41.066564.png)

#### Profiling result for cell 8: 
* 1.34 seconds elapsed
 * 4 output updates


Make all layers visible again:

In [ ]:
%%time

for layer in plot_options.layer.choices:
    plot_options.layer = layer
    plot_options.image_visible = True

![output screenshot](output-003-2024-11-07T20-33-50.703771.png)

#### Profiling result for cell 9: 
* 0.76 seconds elapsed
 * 2 output updates


Increment through colormap upper limits for each layer:

In [ ]:
%%time

for vmax in np.linspace(0, 2**16, 10):
    for layer in plot_options.layer.choices:
        plot_options.layer = layer
        plot_options.stretch_vmin = 0
        plot_options.stretch_vmax = vmax

![output screenshot](output-003-2024-11-07T20-34-07.327145.png)

#### Profiling result for cell 10: 
* 7.21 seconds elapsed
 * 20 output updates
